In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import time

In [ ]:
import ct_projector.projector.numpy as ct_projector
import ct_projector.projector.numpy.fan_equiangluar as ct_fan

In [ ]:
# load a sample CT image
filename = './3.nii.gz'
ct = sitk.ReadImage(filename)
spacing = ct.GetSpacing()
img = sitk.GetArrayFromImage(ct)

# convert image from HU to attenuation coefficient
# This is the approximate relationship
img = (img.astype(np.float32) + 1024) / 1000 * 0.019
img[img < 0] = 0

# also convert to image to our projector dimension batch, z, y, x
img = img[np.newaxis, ...]
img = img[:, ::-1, ...]
spacing = np.array(spacing[::-1])

In [ ]:
# show the ct images
plt.figure(figsize = (12,4))
plt.subplot(131); plt.imshow(img[0, img.shape[1]//2, ...], 'gray', aspect=spacing[1] / spacing[2])
plt.subplot(132); plt.imshow(img[0, :, img.shape[2]//2, :], 'gray', aspect=spacing[0] / spacing[2])
plt.subplot(133); plt.imshow(img[0, ..., img.shape[3]//2], 'gray', aspect=spacing[0] / spacing[1])

In [ ]:
# setup the projector
projector = ct_projector.ct_projector()
projector.from_file('./projector_fan.cfg')
projector.nx = img.shape[3]
projector.ny = img.shape[2]
projector.nz = 1
projector.nv = 1
projector.dx = spacing[2]
projector.dy = spacing[1]
projector.dz = spacing[0]
projector.nview = 768

for k in vars(projector):
    print (k, '=', getattr(projector, k))

In [ ]:
# very important: make sure that the arrays are saved in C order
ct_projector.set_device(0)

angles = projector.get_angles()
# origin_img = img[0, [img.shape[1]//3, img.shape[1]//2, img.shape[1]//3*2], ...]
origin_img = img[0, :64, ...]
origin_img = origin_img[:, np.newaxis, ...]

origin_img = np.copy(origin_img, 'C')
angles = np.copy(angles, 'C')

projector.set_projector(ct_fan.distance_driven_fp, angles = angles)

In [ ]:
# forward projection
start = time.time()
fp = projector.fp(origin_img, angles = angles)
end = time.time()
print (end - start)

In [ ]:
# show the projections
plt.figure(figsize = (8,8))
plt.imshow(fp[1,:, 0, :], 'gray')

In [ ]:
fprj = ct_fan.ramp_filter(projector, fp, filter_type='RL')
recon = ct_fan.fbp_bp(projector, fprj, angles)

In [ ]:
plt.figure(figsize = (16,8))
plt.subplot(121); plt.imshow(recon[2,0, :, :], 'gray', vmin=0, vmax=0.0225)
plt.subplot(122); plt.imshow(origin_img[2,0, :, :], 'gray', vmin=0, vmax=0.0225)

In [ ]:
projector.set_backprojector(ct_fan.distance_driven_bp, angles=angles, is_fbp=True)
recon_dd = projector.bp(fprj)

In [ ]:
plt.figure(figsize = (16,8))
plt.subplot(121); plt.imshow(recon_dd[2,0, :, :], 'gray', vmin=0, vmax=0.0225)
plt.subplot(122); plt.imshow(origin_img[2,0, :, :], 'gray', vmin=0, vmax=0.0225)